# **OJKGPT**

This is the Jupyter Notebook for the development of the 1st version of OJKGPT for the showcase demo capabilites to wealth team program. <br>
In this development, we use several data sources for enriching the RAG system of the chatbot. 

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from utils.models_define import ModelName

# storing or not
STORE = False
DELETE = False
model_name = ModelName.AZURE_OPENAI

## **LLM Models Define**

In [4]:
from llama_index.core import Settings
from utils.models_define import get_llm_and_embedding

# ollama/openai/azure_openai
llm, embedding_llm = get_llm_and_embedding(model_name=model_name)

Settings.llm = llm
Settings.embed_model = embedding_llm

## **Loading Documents**

### **Document Reader and Node Parser**

In [5]:
from utils.documents_reader import read_documents
from utils.node_parser import parse_nodes
from utils.doc_store import store_docstore, load_docstore

path = './data'

if STORE:
    docs_all = read_documents(path=path)
    nodes_all = parse_nodes(documents=docs_all, llm=llm)
    docstore = store_docstore(documents=docs_all, delete=DELETE)
else:
    docstore = load_docstore()

## **Indexing & Storing**

In [6]:
from utils.vector_store import (store_vector_index, load_vector_index)

# store
if STORE:
    index_all = store_vector_index(nodes=nodes_all,embed_model=embedding_llm, delete=DELETE)

# load
else:
    index_all = load_vector_index()

Loading the index completed.


## **Querying**

### **Retriever**

In [7]:
from llama_index.retrievers.bm25 import BM25Retriever
from utils.fusion_retriever import FusionRetriever

vector_retriever_all = index_all.as_retriever(similarity_top_k=5)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore, similarity_top_k=5
)

fusion_retriever = FusionRetriever(
    llm=llm,
    retrievers=[vector_retriever_all, bm25_retriever],
    similarity_top_k=5,
)

### **Query Engine**

In [8]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine_all = RetrieverQueryEngine.from_args(retriever=fusion_retriever)

In [9]:
# a = "Apa judul peraturan 7/33/PBI/2005?" # Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat
b = "Kapan surat edaran No. 15/26/DPbS mulai berlaku?" # 1 Agustus 2013.
# c = "Siapa nama dan jabatannya yang menandatangani surat dengan nomor 1/SEOJK.04/2013?" # NURHAIDA, kepala eksekutif pengawas pasar modal
d = "Saya ingin menyelenggarakan kegiatan pasar modal berikan saya nomor surat yang membahas mengenai hal ini!" # 1/SEOJK.04/2013
e = "Berapa persen jaminan moneter pada tanggal 20 Agustus 1958?" # 7,3%
f = "Surat edaran nomor berapa yang mengatur bank umum syariah dan unit usaha syariah?" # 15/26/DPbS
g = "Apa kepanjangan dari PAPSI?" # Pedoman Akuntansi Perbankan Syariah Indonesia
h = "apa judul peraturan nomor 112/KMK.03/2001?" # Keputusan Menteri Keuangan tentang Pemotongan Pajak Penghasil Pasal 21 atas Penghasilan berupa Uang Pesangon, Uang Tebusan Pensiun, dan Tunjangan Hari Tua atau Jaminan Hari Tua
i = "Saya ingin membuat sistem informasi lembaga jasa keuangan, berikan nomor regulasi dari peraturan yang membahas tentang manejemen risiko nya!" # 4/POJK.05/2021
j = "Apa kepanjangan dari SWDKLLJ?" # Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
k = "Berapa nilai SWDKLLJ dari jeep?" # Rp.140.000
l = "Apa latar belakang dari peraturan NOMOR 4/POJK.05/2021?" # dalam bentuk list
m = "Apa itu LJKNB?" # Lembaga Jasa Keuangan Non Bank
n = "Apakah KMK Nomor 462/KMK.04/1998 masih berlaku" # tidak
o = "Apa itu Uang Pesangon?" # penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan  hubungan kerja, termasuk uang penghargaan masa kerja dan uang  ganti kerugian
p = "Apa itu CKPN?" # Cadangan Kerugian Penurunan Nilai.

z = "What is the limit of QRIS Transaction Nominal?"

query_str = o

In [10]:
response = query_engine_all.query(query_str)

100%|██████████| 6/6 [00:03<00:00,  1.68it/s]


In [11]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** Uang Pesangon adalah penghasilan yang diberikan oleh pemberi kerja kepada karyawan sebagai bentuk kompensasi atas berakhirnya masa kerja atau pemutusan hubungan kerja. Penghasilan ini mencakup uang penghargaan atas masa kerja dan uang ganti kerugian.

## **Build Agent**

In [12]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_tools = [
    QueryEngineTool(
        query_engine=query_engine_all,
        metadata=ToolMetadata(
            name="vector_tool_all",
            description="Useful for retrieving answers from the document store and the vector index.",
        )
    ),
]

In [13]:
system_prompt = """
You are a chatbot able to help answer questions about various types of regulations in Indonesia, including those from OJK, Bapepam, Bank Indonesia, and more.

You MUST reply based on the user's language.
You must ALWAYS use the vector search tool when answering a question.
You are NOT ALLOWED to hallucinate.
You are ONLY ALLOWED to answer the question based on documents that you have been trained on.

These are the regulations available:
- Peraturan OJK
- Surat Edaran OJK
- PPBI
- Klasifikasi Bapepam
- SEBI
- Peraturan Keputusan Menteri
- Peraturan Bapepam
- Undang-Undang
- Peraturan Pemerintah

You are designed to answer queries from the user which includes but is not limited to: Summarizing contents, answering specific articles/sections of the regulation, etc.
You have access to a vector search tool to help you find relevant information in the regulations database.
Given the context information and not prior knowledge, answer the query using the vector search tool provided.
Use the previous chat history, or the context above, to interact and help the user.
----------------------------------------------
Query: {query_str}
Vector Search Results: {query_results}
Answer: {answer}
"""

In [14]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
memory = ChatMemoryBuffer.from_defaults(
    token_limit=10000,
    chat_store=chat_store,
    chat_store_key="user1",
)

In [15]:
# from utils.custom_agent import YourOpenAIAgent, CustomAgent
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent.legacy.react.base import ReActAgent

# agent = custom_agent(
#     llm=llm,
#     query_engine_tools=query_tools,
#     verbose=True,
# )

# agent = YourOpenAIAgent(
#     llm=llm,
#     tools=query_tools,
# )

# agent = CustomAgent(
#     llm=llm,
#     query_engine_tools=query_tools,
#     verbose=True,
# )

agent_openai = OpenAIAgent.from_tools(
    llm=llm,
    tools=query_tools,
    verbose=True,
    memory=memory,
    system_prompt=system_prompt,
)

agent_react = ReActAgent.from_tools(
    llm=llm,
    memory=memory,
    tools=query_tools,
    verbose=True,
    system_prompt=system_prompt,
)

In [16]:
response_openai = agent_openai.chat(query_str)
response_react = agent_react.chat(query_str)

Added user message to memory: Apa itu Uang Pesangon?
Thought: The current language of the user is: Indonesian. I need to use a tool to help me answer the question.
Action: vector_tool_all
Action Input: {'input': 'Apa itu Uang Pesangon?'}


100%|██████████| 6/6 [00:02<00:00,  2.56it/s]


Observation: Uang Pesangon adalah penghasilan yang diberikan oleh pemberi kerja kepada karyawan sebagai bentuk kompensasi atas berakhirnya masa kerja atau pemutusan hubungan kerja. Penghasilan ini mencakup uang penghargaan atas masa kerja dan uang ganti kerugian.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Uang pesangon adalah penghasilan yang diberikan oleh pemberi kerja kepada karyawan sebagai bentuk kompensasi atas berakhirnya masa kerja atau pemutusan hubungan kerja. Penghasilan ini mencakup uang penghargaan atas masa kerja dan uang ganti kerugian.


In [17]:
from llama_index.core.response.notebook_utils import display_response

In [18]:
display_response(response_openai)


**`Final Response:`** Uang pesangon adalah uang yang diberikan kepada pekerja atau karyawan yang dipecat atau mengundurkan diri secara sukarela setelah bekerja dalam suatu perusahaan atau organisasi untuk jangka waktu tertentu. Uang pesangon biasanya diberikan sebagai bentuk kompensasi atas masa kerja yang telah dilakukan oleh pekerja atau karyawan tersebut. Uang pesangon diatur dalam beberapa peraturan di Indonesia, seperti Undang-Undang Ketenagakerjaan dan Peraturan Pemerintah terkait. Apakah ada informasi spesifik yang Anda cari mengenai uang pesangon?

In [19]:
display_response(response_react)


**`Final Response:`** Uang pesangon adalah penghasilan yang diberikan oleh pemberi kerja kepada karyawan sebagai bentuk kompensasi atas berakhirnya masa kerja atau pemutusan hubungan kerja. Penghasilan ini mencakup uang penghargaan atas masa kerja dan uang ganti kerugian.

In [20]:
for node in response_openai.source_nodes:
    print("**********")
    print("id: ", node.id_)
    print("content:\n", node.node.get_content())
    if node.score:
        print("reranking score: ", node.score)
    else:
        print("reranking score: ", "None")
    if node.node.metadata.get("retrieval_score"):
        print("retrieval score: ", node.node.metadata["retrieval_score"])
    else:
        print("retrieval score: ", "None")
    print("**********")
    print("\n")

In [21]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)